In [1]:
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt

import eli5
from eli5.sklearn import PermutationImportance

from pdpbox import pdp

import shap

In [2]:
data = pd.read_csv("SHAP/hospital.csv")

# we want to predict if the patient will readmit
y = data.readmitted

# use all feature columns except target
base_features = [c for c in data.columns if c != "readmitted"]
X = data[base_features]

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
rf_model = RandomForestClassifier(n_estimators=25, random_state=1).fit(train_X, train_y)

In [3]:
# there are many features, use permutation importance to see which feature is important
p = PermutationImportance(rf_model, random_state=1).fit(val_X, val_y)
eli5.show_weights(p, feature_names=val_X.columns.tolist())

Weight,Feature
0.0434 ± 0.0059,number_inpatient
0.0093 ± 0.0054,number_emergency
0.0065 ± 0.0041,number_outpatient
0.0033 ± 0.0056,num_medications
0.0030 ± 0.0014,age_[50-60)
0.0020 ± 0.0041,number_diagnoses
0.0015 ± 0.0046,payer_code_MC
0.0015 ± 0.0028,diabetesMed_Yes
0.0015 ± 0.0013,payer_code_HM
0.0012 ± 0.0016,diag_3_250


In [4]:
# observation
# feature number_inpatient seems important
# use pdp to see how it affects readmitted

In [5]:
feature_name = 'number_inpatient'
pdp_num = pdp.pdp_isolate(model=rf_model, dataset=val_X, model_features=val_X.columns, feature=feature_name)
pdp.pdp_plot(pdp_num, feature_name)
plt.show()

In [6]:
# observation
# greater number_impatient equals greater prediction

In [7]:
# the scale from one pdp is difficult to comprehend
# get pdp for time_in_hospital as a comparison
feature_name = 'time_in_hospital'
pdp_time = pdp.pdp_isolate(model=rf_model, dataset=val_X, model_features=val_X.columns, feature=feature_name)
pdp.pdp_plot(pdp_time, feature_name)
plt.show()

In [8]:
# observation
# time_in_hospital seems neglectable
# but intuition tells us that time_in_hospital should affect readmitted

In [9]:
# check by showing the average readmittion rate for each value of time_in_hospital

# use only the train data
all_train = pd.concat([train_X, train_y], axis=1)

# group readmitted by time_in_hospital
all_train.groupby(['time_in_hospital']).mean().readmitted.plot()
plt.show()

In [10]:
# use SHAP to visualize that, for a sample, show how each feature affect readmitted

# create a function that takes on sample and displays SHAP
def display_shap(model, data_for_prediction):
    # also has explainer for other models such as DeepExplainer, KernelExplainer
    tree_explainer = shap.TreeExplainer(model)

    shap_values = tree_explainer.shap_values(data_for_prediction)
    shap.initjs()

    return shap.force_plot(tree_explainer.expected_value[1], shap_values[1],
                           data_for_prediction)  # [1] means value for true, [0] is false

# test on 1 sample data
sample_data_for_prediction = val_X.iloc[0].astype(float)
display_shap(rf_model, sample_data_for_prediction)


In [12]:
# advanced SHAP value with summary & interaction plot

base_features = ['number_inpatient', 'num_medications', 'number_diagnoses', 'num_lab_procedures',
                 'num_procedures', 'time_in_hospital', 'number_outpatient', 'number_emergency',
                 'gender_Female', 'payer_code_?', 'medical_specialty_?', 'diag_1_428', 'diag_1_414',
                 'diabetesMed_Yes', 'A1Cresult_None']

# shap might have errors if mixing bool and float
X = data[base_features].astype(float)

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

# SHAP may be slow, so as an example use smaller val set
small_val_X = val_X.iloc[:150]

rf_model = RandomForestClassifier(n_estimators=25, random_state=1).fit(train_X, train_y)

data.describe()

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,readmitted
count,25000.000000,25000.00000,25000.000000,25000.000000,25000.000000,25000.000000,25000.00000,25000.000000,25000.000000
mean,4.395640,42.96012,1.341080,15.988440,0.365920,0.203280,0.64300,7.420160,0.456400
std,2.991165,19.76881,1.705398,8.107743,1.224419,0.982973,1.26286,1.940932,0.498105
min,1.000000,1.00000,0.000000,1.000000,0.000000,0.000000,0.00000,1.000000,0.000000
25%,2.000000,31.00000,0.000000,10.000000,0.000000,0.000000,0.00000,6.000000,0.000000
50%,4.000000,44.00000,1.000000,15.000000,0.000000,0.000000,0.00000,8.000000,0.000000
75%,6.000000,57.00000,2.000000,20.000000,0.000000,0.000000,1.00000,9.000000,1.000000
max,14.000000,126.00000,6.000000,81.000000,36.000000,64.000000,21.00000,16.000000,1.000000


In [13]:
# SHAP summary
explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(small_val_X)
shap.summary_plot(shap_values[1], small_val_X)

In [14]:
# observation
# If a feature is widely spread, then it probably has great permutation importance.
# If a feature has colors mixed, i.e. for some positive values of the feature it increase prediction
# but for some it decrease prediction, it is possible that there is some interaction with another feature.
# In this case, we may need SHAP dependency plots

In [15]:
# both num_medications and num_lab_procedures have mixed colors, use SHAP dependency plot

shap.dependence_plot('num_medications', shap_values[1], small_val_X)
shap.dependence_plot('num_lab_procedures', shap_values[1], small_val_X)